In [1]:
!pip -q install transformers datasets tqdm

import torch, sys, transformers, datasets
print("Python:", sys.version)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)


Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
Torch: 2.8.0+cu126 | CUDA: True
Transformers: 4.56.1
Datasets: 4.0.0


In [2]:
MODEL_NAME = "distilbert-base-uncased"
OUT_DIR    = "/content/sarcasm_model"
MAX_LEN    = 128


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

ds = load_dataset("tweet_eval", "irony")  # public, stable schema
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def prep(ex):
    text = (ex.get("text") or "").strip()
    enc  = tok(text, truncation=True, max_length=MAX_LEN)
    enc["labels"] = int(ex["label"])      # 0=non-irony, 1=irony/sarcasm
    return enc

train = ds["train"].map(prep, remove_columns=ds["train"].column_names)
test  = ds["test"].map(prep,  remove_columns=ds["test"].column_names)

print(len(train), len(test), train[0].keys())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

2862 784 dict_keys(['input_ids', 'attention_mask', 'labels'])


In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments
import inspect, torch

args_kwargs = dict(
    output_dir="/content/sarcasm_ckpt",
    per_device_train_batch_size=32 if torch.cuda.is_available() else 16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=100,
    save_strategy="epoch",
    report_to=[]
)
sig = inspect.signature(TrainingArguments.__init__)
if "evaluation_strategy" in sig.parameters:
    args_kwargs["evaluation_strategy"] = "epoch"
else:
    args_kwargs["eval_strategy"] = "epoch"

args = TrainingArguments(**args_kwargs)


In [6]:
from transformers import Trainer
import numpy as np

def metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = (preds == labels).mean()
    return {"accuracy": float(acc)}

trainer = Trainer(
    model=model, args=args, tokenizer=tok,
    train_dataset=train, eval_dataset=test,
    compute_metrics=metrics
)
trainer.train()


/tmp/ipython-input-4071550660.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.653906,0.594388
2,0.649200,0.620838,0.644133
3,0.558100,0.636100,0.645408


TrainOutput(global_step=270, training_loss=0.5773808938485605, metrics={'train_runtime': 64.177, 'train_samples_per_second': 133.786, 'train_steps_per_second': 4.207, 'total_flos': 97118266344336.0, 'train_loss': 0.5773808938485605, 'epoch': 3.0})

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os, shutil

CKPT_DIR = "/content/sarcasm_ckpt"             # your training output_dir
BEST = f"{CKPT_DIR}/checkpoint-180"            # epoch 2 in your run
OUT_DIR = "/content/sarcasm_model"             # final export folder
BASE_MODEL = "distilbert-base-uncased"         # tokenizer source

# 1) Load tokenizer from the base model (or wherever you created `tok` initially)
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# 2) Load the best checkpoint weights
best_model = AutoModelForSequenceClassification.from_pretrained(BEST)

# 3) Save a clean, final model folder to use locally
os.makedirs(OUT_DIR, exist_ok=True)
best_model.save_pretrained(OUT_DIR)
tok.save_pretrained(OUT_DIR)
print("Exported best (epoch 2) to:", OUT_DIR)

# 4) (optional) zip for download
zip_path = shutil.make_archive("/content/sarcasm_model", "zip", OUT_DIR)
print("ZIP:", zip_path)


Exported best (epoch 2) to: /content/sarcasm_model
ZIP: /content/sarcasm_model.zip


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tok = AutoTokenizer.from_pretrained(OUT_DIR, use_fast=True)
mdl = AutoModelForSequenceClassification.from_pretrained(OUT_DIR).eval()

def p_sarcasm(t: str) -> float:
    x = tok(t, return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        return mdl(**x).logits[0].softmax(-1)[1].item()

print(p_sarcasm("wow, great job being late again"))
print(p_sarcasm("meeting at 5 pm, please be on time"))


0.8846350908279419
0.5217200517654419


In [11]:
!pip -q install transformers datasets scikit-learn

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, numpy as np

# Load your saved model folder from Notebook B
MODEL_DIR = "/content/sarcasm_model"   # change if needed
tok = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
mdl = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR).eval()

# Reload the same dataset you trained on
ds = load_dataset("tweet_eval", "irony")  # 0=non-sarcasm, 1=sarcasm
SPLIT = "validation" if "validation" in ds else "test"


In [12]:
from torch.utils.data import DataLoader

def iter_probs(dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    probs, labels = [], []
    for batch in loader:
        texts = batch["text"]
        y = batch["label"]
        enc = tok(list(texts), return_tensors="pt", truncation=True, max_length=128, padding=True)
        with torch.no_grad():
            logits = mdl(**{k:v for k,v in enc.items()}).logits
            p = logits.softmax(-1)[:,1].cpu().numpy()  # prob sarcasm
        probs.extend(p.tolist())
        labels.extend([int(i) for i in y])
    return np.array(probs), np.array(labels)

p, y = iter_probs(ds[SPLIT])
p[:5], y[:5], len(p)


(array([0.20662592, 0.1817428 , 0.71932936, 0.69896692, 0.48461014]),
 array([1, 0, 1, 1, 1]),
 955)

In [13]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, precision_recall_curve

# Sweep thresholds to maximize F1 (good default for binary imbalance)
ts = np.linspace(0.3, 0.8, 51)
best_t = max(ts, key=lambda t: f1_score(y, (p>=t).astype(int)))
pred = (p >= best_t).astype(int)

acc = accuracy_score(y, pred)
f1  = f1_score(y, pred)
roc = roc_auc_score(y, p)
ap  = average_precision_score(y, p)

print(f"Chosen threshold: {best_t:.3f}")
print(f"Accuracy: {acc:.3f}  F1: {f1:.3f}  ROC-AUC: {roc:.3f}  PR-AUC(AP): {ap:.3f}")


Chosen threshold: 0.340
Accuracy: 0.640  F1: 0.680  ROC-AUC: 0.730  PR-AUC(AP): 0.725


In [14]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y, pred))
print(classification_report(y, pred, digits=3))

# Show a few false positives/negatives to sanity check behavior
fp_idx = np.where((pred==1) & (y==0))[0][:5]
fn_idx = np.where((pred==0) & (y==1))[0][:5]

print("\nFalse Positives (pred sarcastic but gold non-sarcastic):")
for i in fp_idx:
    print(f"[p={p[i]:.2f}] {ds[SPLIT][i]['text'][:140]}")

print("\nFalse Negatives (pred non-sarcastic but gold sarcastic):")
for i in fn_idx:
    print(f"[p={p[i]:.2f}] {ds[SPLIT][i]['text'][:140]}")


[[245 254]
 [ 90 366]]
              precision    recall  f1-score   support

           0      0.731     0.491     0.588       499
           1      0.590     0.803     0.680       456

    accuracy                          0.640       955
   macro avg      0.661     0.647     0.634       955
weighted avg      0.664     0.640     0.632       955


False Positives (pred sarcastic but gold non-sarcastic):


TypeError: Wrong key type: '8' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.

In [15]:
from sklearn.metrics import brier_score_loss
brier = brier_score_loss(y, p)  # lower is better
print("Brier score:", round(brier, 4))


Brier score: 0.2109


In [16]:
import numpy as np
def ece(probs, labels, bins=10):
    bins_=np.linspace(0,1,bins+1); ece=0; n=len(probs)
    for i in range(bins):
        lo,hi=bins_[i],bins_[i+1]
        m = (probs>=lo)&(probs<hi)
        if m.any():
            conf = probs[m].mean(); acc = labels[m].mean()
            ece += (m.sum()/n)*abs(acc-conf)
    return ece
print("ECE(10 bins):", round(ece(p,y,10),4))


ECE(10 bins): 0.0567


In [17]:
def text_len_bucket(t):
    L=len(t.split());
    return "short" if L<8 else ("med" if L<18 else "long")

buckets = {"short":[], "med":[], "long":[]}
for i, ex in enumerate(ds[SPLIT]):
    b=text_len_bucket(ex["text"])
    buckets[b].append(i)

for b, idxs in buckets.items():
    if not idxs: continue
    acc_b = accuracy_score(y[idxs], (p[idxs]>=best_t).astype(int))
    f1_b  = f1_score(y[idxs], (p[idxs]>=best_t).astype(int))
    print(f"{b:>5} | n={len(idxs):4d} | acc={acc_b:.3f} | f1={f1_b:.3f}")


short | n= 160 | acc=0.650 | f1=0.696
  med | n= 541 | acc=0.640 | f1=0.661
 long | n= 254 | acc=0.634 | f1=0.707


In [18]:
import json, os
os.makedirs(MODEL_DIR, exist_ok=True)
with open(f"{MODEL_DIR}/agent_meta.json","w") as f:
    json.dump({"sarcasm_threshold": float(best_t)}, f, indent=2)
print("Saved threshold to agent_meta.json")


Saved threshold to agent_meta.json


In [19]:
import time
sample = "wow, great job being late again"
x = tok(sample, return_tensors="pt", truncation=True, max_length=128)
st=time.time()
for _ in range(50):
    with torch.no_grad(): _ = mdl(**x).logits
dt=(time.time()-st)/50
print(f"Avg latency per call: {dt*1000:.1f} ms")


Avg latency per call: 53.1 ms


In [21]:
print(p_sarcasm("kill yourself"))
print(p_sarcasm("meeting at 5 pm"))

0.4103764593601227
0.604198694229126


In [22]:
!pip -q install transformers datasets scikit-learn
import numpy as np, torch, inspect
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)


In [23]:
ds = load_dataset("tweet_eval", "irony")

def preprocess_tweet(t: str) -> str:
    t = t.replace("https://", "http://")  # simple normalization
    # Replace mentions & urls with placeholders
    import re
    t = re.sub(r"@\w+", "@USER", t)
    t = re.sub(r"http\S+|www\.\S+", "URL", t)
    return t

# Peek
print(ds)
print("Example before/after:\n", ds["train"][0]["text"], "\n→", preprocess_tweet(ds["train"][0]["text"]))


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2862
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 784
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 955
    })
})
Example before/after:
 seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life 
→ seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life


In [24]:
MODEL_NAME = "vinai/bertweet-base"  # tweets specialist
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)  # BERTweet often needs slow tokenizer
MAX_LEN = 128

def prep(ex):
    text = preprocess_tweet(ex["text"])
    enc  = tok(text, truncation=True, max_length=MAX_LEN)
    enc["labels"] = int(ex["label"])  # 0=non-sarcastic, 1=sarcastic
    return enc

train = ds["train"].map(prep, remove_columns=ds["train"].column_names)
valid = (ds["validation"] if "validation" in ds else ds["test"]).map(prep, remove_columns=ds["train"].column_names)
test  = ds["test"].map(prep,  remove_columns=ds["test"].column_names)

len(train), len(valid), len(test)


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/955 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

(2862, 955, 784)

In [26]:
!pip3 install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=2159c2c720d91c807e1c318723152f193baec39db86e362bdc5a3da44764a83d
  Stored in directory: /root/.cache/pip/wheels/0d/bf/a2/536017b4a6232aef0fb92831af35facd6590c0af0f3983f63b
Successfully built emoji


In [27]:
MODEL_NAME = "vinai/bertweet-base"  # tweets specialist
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)  # BERTweet often needs slow tokenizer
MAX_LEN = 128

def prep(ex):
    text = preprocess_tweet(ex["text"])
    enc  = tok(text, truncation=True, max_length=MAX_LEN)
    enc["labels"] = int(ex["label"])  # 0=non-sarcastic, 1=sarcastic
    return enc

train = ds["train"].map(prep, remove_columns=ds["train"].column_names)
valid = (ds["validation"] if "validation" in ds else ds["test"]).map(prep, remove_columns=ds["train"].column_names)
test  = ds["test"].map(prep,  remove_columns=ds["test"].column_names)

len(train), len(valid), len(test)


Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/955 [00:00<?, ? examples/s]

Map:   0%|          | 0/784 [00:00<?, ? examples/s]

(2862, 955, 784)

In [28]:
labels_np = np.array(ds["train"]["label"])
n_pos = int((labels_np == 1).sum())
n_neg = int((labels_np == 0).sum())
n_tot = len(labels_np)
class_weights = torch.tensor([n_tot/(2*n_neg), n_tot/(2*n_pos)], dtype=torch.float)
print("class weights:", class_weights.tolist())


class weights: [1.0098800659179688, 0.9903114438056946]


In [29]:
from torch import nn

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(
            weight=self.class_weights.to(logits.device) if self.class_weights is not None else None
        )
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [30]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

args_kwargs = dict(
    output_dir="/content/bertweet_irony_ckpt",
    per_device_train_batch_size=32 if torch.cuda.is_available() else 16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    num_train_epochs=5,        # early stopping will stop earlier if needed
    weight_decay=0.01,
    warmup_ratio=0.06,
    logging_steps=100,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=[]
)
key = "evaluation_strategy" if "evaluation_strategy" in inspect.signature(TrainingArguments.__init__).parameters else "eval_strategy"
args_kwargs[key] = "epoch"
args = TrainingArguments(**args_kwargs)


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# --- WeightedTrainer (v4/v5 compatible) ---
from transformers import Trainer
from torch import nn
import torch, inspect

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    # accept the extra kwarg used in newer HF versions
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        if labels.dtype != torch.long:
            labels = labels.long()
        loss_fct = nn.CrossEntropyLoss(
            weight=self.class_weights.to(logits.device) if self.class_weights is not None else None
        )
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# --- Build the trainer again with the right arg name for tokenizer/processing_class ---
from transformers import EarlyStoppingCallback

tok_arg = "processing_class" if "processing_class" in inspect.signature(Trainer.__init__).parameters else "tokenizer"

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=valid,
    compute_metrics=metrics,
    class_weights=class_weights,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    **{tok_arg: tok},  # passes either processing_class=tok (v5) or tokenizer=tok (v4)
)

trainer.train()
print("Best checkpoint:", trainer.state.best_model_checkpoint)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.568416,0.696335,0.726415
2,0.634000,0.519318,0.748691,0.745763
3,0.456400,0.561190,0.744503,0.757937
4,0.324400,0.645926,0.750785,0.760563
5,0.220700,0.619936,0.763351,0.759574


Best checkpoint: /content/bertweet_irony_ckpt/checkpoint-360


In [33]:
OUT_DIR = "/content/sarcasm_bertweet_model"
trainer.model.save_pretrained(OUT_DIR)
tok.save_pretrained(OUT_DIR)
print("Saved:", OUT_DIR)


Saved: /content/sarcasm_bertweet_model


In [35]:
import torch, numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

device = next(trainer.model.parameters()).device
split = ds["validation"] if "validation" in ds else ds["test"]

def collate(batch):
    texts  = [preprocess_tweet(ex["text"]) for ex in batch]
    labels = torch.tensor([int(ex["label"]) for ex in batch], dtype=torch.long)
    enc = tok(texts, return_tensors="pt", truncation=True, max_length=MAX_LEN, padding=True)
    enc["labels"] = labels
    return enc

loader = DataLoader(split, batch_size=128, shuffle=False, collate_fn=collate)

probs, gold = [], []
trainer.model.eval()
with torch.no_grad():
    for batch in loader:
        labels = batch.pop("labels")
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = trainer.model(**batch).logits
        p = torch.softmax(logits, dim=-1)[:, 1].detach().cpu().numpy()
        probs.extend(p.tolist())
        gold.extend(labels.numpy().tolist())

probs = np.array(probs); gold = np.array(gold)

def balanced_acc_at(t):
    pred = (probs >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(gold, pred).ravel()
    tpr = tp / max(tp+fn, 1)      # recall positive
    tnr = tn / max(tn+fp, 1)      # recall negative (specificity)
    return 0.5*(tpr + tnr)

ts = np.linspace(0.3, 0.8, 101)
best_t = max(ts, key=balanced_acc_at)
print("best_threshold_for_balanced_acc:", round(float(best_t), 3))

# save for agent
import json, os
with open(f"{OUT_DIR}/agent_meta.json", "w") as f:
    json.dump({"sarcasm_threshold": float(best_t)}, f, indent=2)
print("Saved threshold to agent_meta.json")


best_threshold_for_balanced_acc: 0.78
Saved threshold to agent_meta.json


In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tok2 = AutoTokenizer.from_pretrained(OUT_DIR, use_fast=False)
mdl2 = AutoModelForSequenceClassification.from_pretrained(OUT_DIR).eval()

def p_sarcasm_bertweet(text: str) -> float:
    x = tok2(preprocess_tweet(text), return_tensors="pt", truncation=True, max_length=128)
    with torch.no_grad():
        return mdl2(**x).logits[0].softmax(-1)[1].item()

print(p_sarcasm_bertweet("wow, great job being late again"))
print(p_sarcasm_bertweet("meeting at 6 pm please be on time"))


0.9684514403343201
0.9400405883789062


In [43]:
# === Confusion matrix for current sarcasm model ===
import numpy as np, torch, os, json, inspect, re
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, classification_report

# 0) Model/tokenizer + device
try:
    mdl = trainer.model.eval()
    tok_used = tok
except NameError:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    OUT_DIR = "/content/sarcasm_bertweet_model"  # <-- change if needed
    tok_used = AutoTokenizer.from_pretrained(OUT_DIR, use_fast=False)
    mdl = AutoModelForSequenceClassification.from_pretrained(OUT_DIR).eval()

device = next(mdl.parameters()).device

# 1) Dataset split
split = ds["validation"] if "validation" in ds else ds["test"]

# 2) Preprocess (robust even if preprocess_tweet isn't defined)
def preprocess_tweet_safe(t: str) -> str:
    try:
        return preprocess_tweet(t)
    except NameError:
        t = t.replace("https://", "http://")
        t = re.sub(r"@\w+", "@USER", t)
        t = re.sub(r"http\S+|www\.\S+", "URL", t)
        return t

MAX_LEN = 128 if 'MAX_LEN' not in globals() else MAX_LEN

def collate(batch):
    texts  = [preprocess_tweet_safe(ex["text"]) for ex in batch]
    labels = torch.tensor([int(ex["label"]) for ex in batch], dtype=torch.long)
    enc = tok_used(texts, return_tensors="pt", truncation=True, max_length=MAX_LEN, padding=True)
    enc["labels"] = labels
    return enc

loader = DataLoader(split, batch_size=128, shuffle=False, collate_fn=collate)

# 3) Collect probabilities and gold labels
probs, gold = [], []
mdl.eval()
with torch.no_grad():
    for batch in loader:
        labels = batch.pop("labels")
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = mdl(**batch).logits
        p = torch.softmax(logits, dim=-1)[:, 1].detach().cpu().numpy()
        probs.extend(p.tolist())
        gold.extend(labels.numpy().tolist())

probs = np.array(probs); gold = np.array(gold)

# 4) Pick threshold: use saved/previous best_t if available, else maximize balanced accuracy
def balanced_acc_at(t):
    pred = (probs >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(gold, pred).ravel()
    tpr = tp / max(tp+fn, 1)   # recall +
    tnr = tn / max(tn+fp, 1)   # specificity
    return 0.5*(tpr + tnr)

try:
    t = float(best_t)
except NameError:
    ts = np.linspace(0.3, 0.99, 140)
    t = max(ts, key=balanced_acc_at)

print(f"Using threshold: {t:.3f}")

# 5) Confusion matrix + metrics
pred = (probs >= t).astype(int)
tn, fp, fn, tp = confusion_matrix(gold, pred).ravel()
acc = accuracy_score(gold, pred)
prec, rec, f1, _ = precision_recall_fscore_support(gold, pred, average='binary', zero_division=0)
spec = tn / max(tn+fp, 1)
bacc = 0.5*(rec + spec)

print("\nConfusion matrix [[TN FP],[FN TP]]:")
print(np.array([[tn, fp],[fn, tp]]))
print(f"\nAccuracy       : {acc:.3f}")
print(f"Precision (pos): {prec:.3f}")
print(f"Recall   (pos) : {rec:.3f}")
print(f"F1       (pos) : {f1:.3f}")
print(f"Specificity    : {spec:.3f}")
print(f"Balanced Acc   : {bacc:.3f}")

# Optional: full per-class report
print("\nClassification report:")
print(classification_report(gold, pred, digits=3))


Using threshold: 0.780

Confusion matrix [[TN FP],[FN TP]]:
[[374 125]
 [104 352]]

Accuracy       : 0.760
Precision (pos): 0.738
Recall   (pos) : 0.772
F1       (pos) : 0.755
Specificity    : 0.749
Balanced Acc   : 0.761

Classification report:
              precision    recall  f1-score   support

           0      0.782     0.749     0.766       499
           1      0.738     0.772     0.755       456

    accuracy                          0.760       955
   macro avg      0.760     0.761     0.760       955
weighted avg      0.761     0.760     0.760       955



In [44]:
SAR_DIR = "/content/sarcasm_bertweet_model"   # or your OUT_DIR


In [45]:
from google.colab import files
zip_path = shutil.make_archive(SAR_DIR, "zip", SAR_DIR)
files.download(zip_path)   # downloads to your laptop


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>